# Summary
- mincorr and minlinreg are similar. maxcorr and maxlinreg are similar. 
- spearman among different layers is lower (even goes negative) in lower layers for:
    - bert_large
    - openai
    - bert_base
  
  but not true for elmo or calypso
- intermediate layers appear most similar to the next and previous layer of the same model
- lower layers (eg. in bert) are more correlated with lower layers in other models

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from os.path import basename, dirname
import torch

In [2]:
# Set `res_d`, `network_l`, `num_neurons_d`
base = "/data/sls/temp/johnmwu/contextual-corr-analysis/results8_"
res_fname = {method : base + method for method in 
                {"maxcorr", "mincorr", "maxlinreg", "minlinreg", "cca", "lincka"}}

res_d = {}
res_d["maxcorr"] = json.load(open(res_fname["maxcorr"], "r"))
res_d["mincorr"] = json.load(open(res_fname["mincorr"], "r"))
res_d["maxlinreg"] = json.load(open(res_fname["maxlinreg"], "r"))
res_d["minlinreg"] = json.load(open(res_fname["minlinreg"], "r"))

network_l = [network for network in res_d["maxcorr"]]

num_neurons_d = {}
for network in network_l:
    num_neurons_d[network] = len(res_d["maxcorr"][network])

# MaxCorr, MinCorr, MaxLinReg, MinLinReg

### Sorts

In [3]:
rk_methods = {"maxcorr", "mincorr", "maxlinreg", "minlinreg"}

In [4]:
neuron_sorts = {network: {} for network in network_l}
for network in network_l:
    for method in rk_methods:
        neuron_sorts[network][method] = [neuron[0] for neuron in res_d[method][network]]

In [5]:
# Set `rk_ar_d`, `rk_df_d`, `spearman_d`
rk_ar_d = {}
rk_df_d = {}
spearman_d = {}
for network in network_l:
    # rk_ar[method_ix, neuron] = rank
    # need to invert the permutation
    rk_ar = np.stack([
            [neuron_sorts[network][method].index(neuron) for neuron in range(len(neuron_sorts[network][method]))]
                      for method in rk_methods
    ]).T
    rk_df = pd.DataFrame(rk_ar, columns=list(rk_methods))
    
    rk_ar_d[network] = rk_ar
    rk_df_d[network] = rk_df
    spearman_d[network] = rk_df.corr()

In [6]:
for network in network_l:
    print(network)
    # print(rk_df_d[network])
    print(spearman_d[network])

bert_large_cased_0
           minlinreg   mincorr   maxcorr  maxlinreg
minlinreg   1.000000  0.841917 -0.205201   0.267656
mincorr     0.841917  1.000000 -0.265457   0.176797
maxcorr    -0.205201 -0.265457  1.000000   0.648580
maxlinreg   0.267656  0.176797  0.648580   1.000000
bert_large_cased_1
           minlinreg   mincorr   maxcorr  maxlinreg
minlinreg   1.000000  0.744525  0.215490   0.278610
mincorr     0.744525  1.000000 -0.017512   0.035103
maxcorr     0.215490 -0.017512  1.000000   0.966639
maxlinreg   0.278610  0.035103  0.966639   1.000000
bert_large_cased_2
           minlinreg   mincorr   maxcorr  maxlinreg
minlinreg   1.000000  0.739880  0.435666   0.452293
mincorr     0.739880  1.000000  0.115634   0.131480
maxcorr     0.435666  0.115634  1.000000   0.977871
maxlinreg   0.452293  0.131480  0.977871   1.000000
bert_large_cased_3
           minlinreg   mincorr   maxcorr  maxlinreg
minlinreg   1.000000  0.799138  0.348838   0.376619
mincorr     0.799138  1.000000  0.098167

           minlinreg   mincorr   maxcorr  maxlinreg
minlinreg   1.000000  0.828760  0.500716   0.682169
mincorr     0.828760  1.000000  0.335521   0.504240
maxcorr     0.500716  0.335521  1.000000   0.758273
maxlinreg   0.682169  0.504240  0.758273   1.000000
xlnet_large_cased_23
           minlinreg   mincorr   maxcorr  maxlinreg
minlinreg   1.000000  0.803794  0.452170   0.614470
mincorr     0.803794  1.000000  0.309494   0.424558
maxcorr     0.452170  0.309494  1.000000   0.704952
maxlinreg   0.614470  0.424558  0.704952   1.000000


### Distributions

In [7]:
for network in network_l:
    print("\n\n" + network + ":")
    for method in rk_methods:
        print("\n" + method)
        corr_ar = np.stack([
            list(notated_neuron[1].values())
            for notated_neuron in res_d[method][network]
        ])
        corr_df = pd.DataFrame(np.abs(corr_ar),
                              columns=[net for net in network_l if net != network])
        
        print("\nmaxcounts")
        print(corr_df.idxmax(axis=1).value_counts())
        
        print("\nmincounts")
        print(corr_df.idxmin(axis=1).value_counts())
        
        print("\ncorr_df_pctl")
        corr_df_pctl = pd.DataFrame()
        corr_df_pctl["max"] = corr_df.max(axis=1)
        corr_df_pctl["min"] = corr_df.min(axis=1)
        corr_df_pctl["med"] = corr_df.median(axis=1)
        print(corr_df_pctl.rank().corr()) # spearman
        
#         corr_df.plot()
#         plt.xlabel("neuron (sorted order)")
#         plt.ylabel("correlation")
#         plt.show()
        
#         plt.hist(corr_ar.ravel(), bins=100)
#         plt.xlabel("correlation")
#         plt.ylabel("count")
#         plt.show()



bert_large_cased_0:

minlinreg

maxcounts
bert_large_cased_1    1021
bert_base_cased_0        3
dtype: int64

mincounts
xlnet_large_cased_23     1011
xlnet_large_cased_0         8
openai_transformer_12       4
elmo_original_0             1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.267658  0.390258
min  0.267658  1.000000  0.936902
med  0.390258  0.936902  1.000000

mincorr

maxcounts
bert_large_cased_1                  981
bert_base_cased_0                    40
calypso_transformer_6_512_base_0      2
openai_transformer_2                  1
dtype: int64

mincounts
xlnet_large_cased_23    687
elmo_4x4096_512_2        67
xlnet_large_cased_20     50
xlnet_large_cased_22     34
xlnet_large_cased_16     32
xlnet_large_cased_19     26
xlnet_large_cased_21     20
xlnet_large_cased_17     19
xlnet_large_cased_18     19
xlnet_large_cased_15     18
elmo_4x4096_512_3         9
xlnet_large_cased_13      9
xlnet_large_cased_14      4
xlnet_large_cased_12      3

xlnet_large_cased_23                574
xlnet_large_cased_22                 65
xlnet_large_cased_20                 48
elmo_4x4096_512_2                    43
xlnet_large_cased_15                 31
xlnet_large_cased_21                 30
xlnet_large_cased_16                 29
elmo_4x4096_512_3                    27
xlnet_large_cased_19                 26
xlnet_large_cased_13                 24
xlnet_large_cased_18                 21
xlnet_large_cased_14                 18
xlnet_large_cased_17                 14
xlnet_large_cased_12                  8
elmo_4x4096_512_4                     6
xlnet_large_cased_11                  5
calypso_transformer_6_512_base_6      4
elmo_4x4096_512_0                     4
bert_base_cased_12                    4
xlnet_large_cased_8                   4
xlnet_large_cased_9                   3
xlnet_large_cased_10                  3
xlnet_large_cased_3                   3
elmo_original_1                       3
bert_large_cased_24                   3


bert_large_cased_6    1024
dtype: int64

mincounts
xlnet_large_cased_23                673
openai_transformer_12               281
elmo_4x4096_512_0                    30
calypso_transformer_6_512_base_0     20
elmo_original_0                      13
openai_transformer_0                  3
xlnet_large_cased_0                   3
bert_base_cased_0                     1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.509160  0.744116
min  0.509160  1.000000  0.905967
med  0.744116  0.905967  1.000000


bert_large_cased_6:

minlinreg

maxcounts
bert_large_cased_7    1023
bert_large_cased_5       1
dtype: int64

mincounts
xlnet_large_cased_23                497
openai_transformer_12               424
elmo_4x4096_512_0                    53
calypso_transformer_6_512_base_0     25
elmo_original_0                      19
openai_transformer_0                  3
xlnet_large_cased_0                   3
dtype: int64

corr_df_pctl
          max       min       med
max

          max       min       med
max  1.000000  0.580289  0.773440
min  0.580289  1.000000  0.917446
med  0.773440  0.917446  1.000000


bert_large_cased_8:

minlinreg

maxcounts
bert_large_cased_9    1005
bert_large_cased_7      19
dtype: int64

mincounts
openai_transformer_12               658
xlnet_large_cased_23                287
elmo_4x4096_512_0                    33
calypso_transformer_6_512_base_0     31
elmo_original_0                      13
openai_transformer_0                  1
xlnet_large_cased_0                   1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.612203  0.809070
min  0.612203  1.000000  0.921645
med  0.809070  0.921645  1.000000

mincorr

maxcounts
bert_large_cased_7                  612
bert_large_cased_9                  411
calypso_transformer_6_512_base_0      1
dtype: int64

mincounts
xlnet_large_cased_23                616
elmo_4x4096_512_2                    46
xlnet_large_cased_22                 40
elmo_4x4096_512


maxcounts
bert_large_cased_10    1022
bert_large_cased_8        2
dtype: int64

mincounts
openai_transformer_12               719
xlnet_large_cased_23                237
elmo_4x4096_512_0                    41
calypso_transformer_6_512_base_0     19
elmo_original_0                       7
openai_transformer_11                 1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.617152  0.816005
min  0.617152  1.000000  0.923290
med  0.816005  0.923290  1.000000


bert_large_cased_10:

minlinreg

maxcounts
bert_large_cased_11    619
bert_large_cased_9     405
dtype: int64

mincounts
openai_transformer_12               753
xlnet_large_cased_23                200
elmo_4x4096_512_0                    37
calypso_transformer_6_512_base_0     19
elmo_original_0                      12
openai_transformer_0                  3
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.637688  0.825095
min  0.637688  1.000000  0.927000
med  0.825095 

...

elmo_original_0                     106
elmo_4x4096_512_0                    86
elmo_4x4096_512_2                    72
openai_transformer_0                 62
bert_large_cased_5                   57
bert_large_cased_4                   48
bert_large_cased_3                   46
elmo_4x4096_512_3                    44
bert_large_cased_2                   44
bert_large_cased_1                   34
bert_base_cased_1                    32
bert_base_cased_2                    31
openai_transformer_1                 27
bert_base_cased_0                    27
bert_base_cased_12                   24
openai_transformer_2                 22
bert_large_cased_0                   21
bert_large_cased_6                   19
bert_large_cased_23                  18
calypso_transformer_6_512_base_0     18
bert_large_cased_22                  16
bert_large_cased_24                  12
openai_transformer_12                12
bert_large_cased_9                   12
bert_large_cased_8                   10


elmo_original_0                     94
elmo_4x4096_512_0                   89
elmo_4x4096_512_2                   86
bert_large_cased_2                  70
elmo_4x4096_512_3                   53
openai_transformer_0                52
bert_large_cased_3                  49
bert_large_cased_1                  42
bert_large_cased_5                  41
bert_base_cased_1                   39
bert_large_cased_4                  35
bert_base_cased_0                   30
bert_base_cased_12                  23
bert_large_cased_23                 21
bert_large_cased_6                  20
bert_large_cased_0                  18
calypso_transformer_6_512_base_0    16
bert_base_cased_2                   16
bert_large_cased_24                 16
bert_large_cased_9                  16
bert_base_cased_3                   13
bert_large_cased_7                  12
bert_large_cased_22                 12
openai_transformer_1                10
bert_large_cased_8                   9
openai_transformer_12    

elmo_4x4096_512_2                   110
elmo_4x4096_512_0                    77
elmo_original_0                      76
bert_large_cased_2                   60
openai_transformer_0                 51
bert_large_cased_5                   50
elmo_4x4096_512_3                    47
bert_large_cased_3                   46
bert_large_cased_4                   37
bert_large_cased_1                   37
bert_base_cased_1                    36
bert_base_cased_0                    25
bert_base_cased_12                   24
openai_transformer_1                 23
bert_large_cased_0                   22
bert_large_cased_7                   21
bert_large_cased_22                  19
bert_base_cased_2                    16
bert_large_cased_23                  16
calypso_transformer_6_512_base_0     13
bert_large_cased_24                  13
elmo_4x4096_512_1                    13
bert_large_cased_21                  12
bert_large_cased_6                   10
bert_large_cased_9                   10


xlnet_large_cased_20    594
xlnet_large_cased_22    429
xlnet_large_cased_0       1
dtype: int64

mincounts
elmo_4x4096_512_2                   129
elmo_original_0                      78
elmo_4x4096_512_3                    68
openai_transformer_0                 67
elmo_4x4096_512_0                    60
bert_large_cased_5                   44
bert_large_cased_2                   41
bert_large_cased_1                   37
bert_large_cased_4                   34
bert_base_cased_12                   31
bert_large_cased_3                   28
bert_large_cased_6                   25
openai_transformer_1                 24
bert_base_cased_1                    22
bert_base_cased_2                    21
elmo_4x4096_512_1                    20
bert_base_cased_0                    19
bert_large_cased_24                  15
bert_large_cased_0                   13
calypso_transformer_6_512_base_0     12
bert_base_cased_3                    12
bert_large_cased_23                  12
bert_large_c

          max       min       med
max  1.000000  0.309494  0.271656
min  0.309494  1.000000  0.920465
med  0.271656  0.920465  1.000000

maxlinreg

maxcounts
xlnet_large_cased_22    1024
dtype: int64

mincounts
elmo_4x4096_512_0                   631
calypso_transformer_6_512_base_0    169
elmo_original_0                     122
openai_transformer_0                 68
openai_transformer_12                23
bert_base_cased_0                    11
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.614468  0.689218
min  0.614468  1.000000  0.914604
med  0.689218  0.914604  1.000000


In [8]:
network = "calypso_transformer_6_512_base_0"
method = "maxcorr"
print("\n\n" + network + ":")

print("\n" + method)
corr_ar = np.stack([
    list(notated_neuron[1].values())
    for notated_neuron in res_d[method][network]
])
corr_df = pd.DataFrame(np.abs(corr_ar),
                      columns=[net for net in network_l if net != network])

print("\nmaxcounts")
print(corr_df.idxmax(axis=1).value_counts())

print("\nmincounts")
print(corr_df.idxmin(axis=1).value_counts())

print("\ncorr_df_pctl")
corr_df_pctl = pd.DataFrame()
corr_df_pctl["max"] = corr_df.max(axis=1)
corr_df_pctl["min"] = corr_df.min(axis=1)
corr_df_pctl["med"] = corr_df.median(axis=1)
print(corr_df_pctl.rank().corr()) # spearman



calypso_transformer_6_512_base_0:

maxcorr

maxcounts
calypso_transformer_6_512_base_1    914
xlnet_large_cased_0                  32
elmo_original_1                      26
calypso_transformer_6_512_base_4     10
calypso_transformer_6_512_base_2     10
calypso_transformer_6_512_base_6      6
bert_large_cased_20                   6
elmo_4x4096_512_0                     4
calypso_transformer_6_512_base_3      4
xlnet_large_cased_1                   4
bert_large_cased_2                    2
openai_transformer_2                  2
bert_large_cased_5                    2
bert_base_cased_1                     2
dtype: int64

mincounts
xlnet_large_cased_23     530
xlnet_large_cased_22      72
xlnet_large_cased_20      60
xlnet_large_cased_15      38
bert_large_cased_24       32
xlnet_large_cased_16      30
xlnet_large_cased_19      28
xlnet_large_cased_17      26
xlnet_large_cased_18      26
xlnet_large_cased_21      24
xlnet_large_cased_14      20
openai_transformer_12     16
bert_base_ca

In [9]:
corr_df

,bert_large_cased_0,bert_large_cased_1,bert_large_cased_2,bert_large_cased_3,bert_large_cased_4,bert_large_cased_5,bert_large_cased_6,bert_large_cased_7,bert_large_cased_8,bert_large_cased_9,...,xlnet_large_cased_14,xlnet_large_cased_15,xlnet_large_cased_16,xlnet_large_cased_17,xlnet_large_cased_18,xlnet_large_cased_19,xlnet_large_cased_20,xlnet_large_cased_21,xlnet_large_cased_22,xlnet_large_cased_23
0,1.092121,0.581729,0.631106,0.640960,0.764954,0.942338,0.995340,0.991872,1.060557,1.019493,...,0.582762,0.660581,0.719581,0.694891,0.550077,0.814799,0.817463,1.061736,0.887739,0.994141
1,1.092121,0.581729,0.631106,0.640960,0.764954,0.942338,0.995340,0.991872,1.060557,1.019493,...,0.582762,0.660581,0.719581,0.694891,0.550077,0.814799,0.817463,1.061736,0.887739,0.994141
2,0.379900,0.351853,0.344071,0.359065,0.365347,0.366619,0.362762,0.370977,0.362589,0.353846,...,0.330183,0.311211,0.305510,0.259996,0.285883,0.286926,0.283676,0.283785,0.236602,0.220071
3,0.379900,0.351853,0.344071,0.359065,0.365347,0.366619,0.362762,0.370977,0.362589,0.353846,...,0.330183,0.311211,0.305510,0.259996,0.285883,0.286926,0.283676,0.283785,0.236602,0.220071
4,0.444007,0.469004,0.465065,0.476677,0.463614,0.463544,0.471434,0.455668,0.441191,0.447109,...,0.391949,0.383936,0.356506,0.339690,0.341840,0.292611,0.287386,0.296415,0.291305,0.263384
5,0.444007,0.469004,0.465065,0.476677,0.463614,0.463544,0.471434,0.455668,0.441191,0.447109,...,0.391949,0.383936,0.356506,0.339690,0.341840,0.292611,0.287386,0.296415,0.291305,0.263384
6,0.340675,0.249939,0.236870,0.249642,0.280173,0.283287,0.274723,0.318427,0.278526,0.294841,...,0.240269,0.243965,0.244696,0.242097,0.234752,0.197716,0.191882,0.191571,0.175131,0.147953
7,0.340675,0.249939,0.236870,0.249642,0.280173,0.283287,0.274723,0.318427,0.278526,0.294841,...,0.240269,0.243965,0.244696,0.242097,0.234752,0.197716,0.191882,0.191571,0.175131,0.147953
8,0.392372,0.388495,0.395333,0.408167,0.400748,0.406802,0.419341,0.413933,0.404435,0.411815,...,0.312244,0.312505,0.300709,0.288311,0.281828,0.226842,0.222380,0.234031,0.233607,0.225460
9,0.392372,0.388495,0.395333,0.408167,0.400748,0.406802,0.419341,0.413933,0.404435,0.411815,...,0.312244,0.312505,0.300709,0.288311,0.281828,0.226842,0.222380,0.234031,0.233607,0.225460


# LinCKA

In [10]:
cka_d = torch.load(res_fname["lincka"])
df = pd.DataFrame(cka_d)

In [11]:
network = "bert_large_cased_0"
df.sort_values(by=network)[network]

xlnet_large_cased_23                0.202053
xlnet_large_cased_22                0.351866
bert_large_cased_23                 0.354740
calypso_transformer_6_512_base_6    0.355555
bert_large_cased_21                 0.361211
bert_large_cased_24                 0.367523
bert_large_cased_22                 0.375663
xlnet_large_cased_20                0.389854
xlnet_large_cased_16                0.395993
xlnet_large_cased_17                0.397924
xlnet_large_cased_19                0.399624
xlnet_large_cased_18                0.403090
xlnet_large_cased_21                0.404775
bert_base_cased_12                  0.405538
xlnet_large_cased_15                0.407685
openai_transformer_12               0.414044
xlnet_large_cased_14                0.421196
elmo_4x4096_512_0                   0.429179
openai_transformer_4                0.429186
xlnet_large_cased_11                0.437323
calypso_transformer_6_512_base_2    0.437741
openai_transformer_3                0.439869
xlnet_larg

# CCA

In [12]:
cca_d = torch.load(res_fname["cca"])

In [13]:
corrs, sv_similarities, pw_similarities = cca_d['corrs'], cca_d['sv_similarities'], cca_d['pw_similarities']

In [14]:
sv_sim_df = pd.DataFrame(sv_similarities)
pw_sim_df = pd.DataFrame(pw_similarities)
corrs_df = pd.DataFrame(corrs)
maxcorrs_df = corrs_df.applymap(lambda t: float('nan') if type(t) is float else t.max().item())

### SVCCA

In [15]:
network = "bert_large_cased_0"
sv_sim_df.sort_values(by=network)[network]

xlnet_large_cased_23                0.248974
xlnet_large_cased_22                0.252393
xlnet_large_cased_20                0.253511
xlnet_large_cased_19                0.255059
xlnet_large_cased_21                0.255176
xlnet_large_cased_17                0.255192
xlnet_large_cased_18                0.255835
xlnet_large_cased_16                0.257902
xlnet_large_cased_15                0.261066
xlnet_large_cased_14                0.266817
xlnet_large_cased_13                0.274012
elmo_4x4096_512_4                   0.277943
xlnet_large_cased_12                0.286188
openai_transformer_12               0.291095
calypso_transformer_6_512_base_6    0.291605
elmo_original_2                     0.291956
xlnet_large_cased_11                0.296477
bert_large_cased_24                 0.297135
openai_transformer_11               0.305676
xlnet_large_cased_10                0.308109
bert_base_cased_12                  0.315521
bert_large_cased_23                 0.316317
calypso_tr

### PWCCA

In [16]:
network = "bert_large_cased_0"
pw_sim_df.sort_values(by=network)[network]

xlnet_large_cased_23                0.335873
xlnet_large_cased_22                0.345072
xlnet_large_cased_20                0.349186
xlnet_large_cased_21                0.349359
xlnet_large_cased_19                0.352253
xlnet_large_cased_17                0.352838
xlnet_large_cased_18                0.353646
xlnet_large_cased_16                0.355094
xlnet_large_cased_15                0.358623
xlnet_large_cased_14                0.364903
elmo_4x4096_512_4                   0.370091
xlnet_large_cased_13                0.372837
elmo_original_2                     0.384753
xlnet_large_cased_12                0.385278
openai_transformer_12               0.388729
xlnet_large_cased_11                0.395892
calypso_transformer_6_512_base_6    0.396797
bert_large_cased_24                 0.406230
xlnet_large_cased_10                0.407386
openai_transformer_11               0.407604
elmo_4x4096_512_3                   0.415548
xlnet_large_cased_9                 0.417097
openai_tra